## Making images to validate gating



In [ ]:
import numpy as np
import pandas as pd
import os
import re
import cellpose
import tifffile
from cellpose import plot
import matplotlib.pyplot as plt
%matplotlib inline

## Functions

In [ ]:
def centroid_cell_type_mask(quant_df, mask):
    ## Plotting square for each centroid 
    centroid_mask = np.zeros(mask.shape, dtype= "uint16")

    for X_centroid, Y_centroid, cell_id in zip(quant_df["X_centroid"], quant_df["Y_centroid"], quant_df["CellID"]):
        ## Can only use integers to index
        X_centroid = round(X_centroid)
        Y_centroid = round(Y_centroid)

        centroid_mask[Y_centroid-20:Y_centroid+20, X_centroid-20:X_centroid+20] = cell_id

    return centroid_mask

def gating_overlay_wrapper(img_dir, gating_name, cell_type, mask_name, img_name,  
                           subset = True):
    
    ## Load gating data
    quant_df = pd.read_csv(os.path.join(img_dir, gating_name))
    quant_df = quant_df[quant_df["cell_type"] == cell_type]
    print("Load_mask")
    mask = tifffile.imread(os.path.join(img_dir, mask_name))

    print("Make centroid mask")
    centroid_mask = centroid_cell_type_mask(quant_df= quant_df, mask = mask)
    tifffile.imwrite(os.path.join(img_dir, "centroid_mask.tif"), centroid_mask)

    print("Converting mask to binary and shrinking to uint8")
    centroid_mask[centroid_mask > 0] = 255
    centroid_mask = centroid_mask.astype("uint8")
    tifffile.imwrite(os.path.join(img_dir, cell_type + "_binary_centroid_mask.tif"), centroid_mask)
    
    print("Overlaying centroids on markers")
    img = tifffile.imread(os.path.join(img_dir, img_name))
    centroid_mask = centroid_mask[np.newaxis,...]
    region_w_centroids = np.concatenate((img, centroid_mask))

    tifffile.imwrite(os.path.join(img_dir, cell_type + "_img_w_centroids.ome.tif"), region_w_centroids)

    if(subset):
        print("Writing 50% subsetted image")
        upper_bound = round(min(centroid_mask.shape[1:3])/2)
        region_w_centroids = region_w_centroids[:, :, 1_000:upper_bound]
        # region_w_centroids = region_w_centroids[:, 4_000:8_000, 4_000:8_000] # I used this for CD14_A
        tifffile.imwrite(os.path.join(img_dir, cell_type + "_subset_region_w_centroids.ome.tif"), region_w_centroids)

    return None

## Running images

In [ ]:
raw_dir="/stor/scratch/Ehrlich/Users/John/histocytometry/raw_images/images_2023-08-10"
img_dirs= [os.path.join(raw_dir, img_dir) for img_dir in os.listdir(raw_dir) if re.search("_[A-D]$", img_dir) and not re.search("_Sirpa_C$|CD207_CD11c_XCR1_C$", img_dir)]

In [ ]:
img_dir = os.path.join(raw_dir, "20x_pan_DAPI_B220_CD11c_SiglecH_A")

gating_overlay_wrapper(img_dir     = img_dir, 
                       gating_name = "combined_gated_cell_types.csv", 
                       cell_type   = "pDC", 
                       mask_name   = "cp2_mask.tif", 
                       img_name    = "reordered_image_w_thymus_regions_50um.ome.tif", 
                       subset      = True)

In [ ]:
img_dir = os.path.join(raw_dir, "20x_pan_DAPI_B220_CD11c_SiglecH_B")

gating_overlay_wrapper(img_dir     = img_dir, 
                       gating_name = "combined_gated_cell_types.csv", 
                       cell_type   = "pDC", 
                       mask_name   = "cp2_mask.tif", 
                       img_name    = "reordered_image_w_thymus_regions_50um.ome.tif", 
                       subset      = True)

In [ ]:
img_dir = os.path.join(raw_dir, "20x_pan_DAPI_B220_CD11c_SiglecH_C")

gating_overlay_wrapper(img_dir     = img_dir, 
                       gating_name = "combined_gated_cell_types.csv", 
                       cell_type   = "pDC", 
                       mask_name   = "cp2_mask.tif", 
                       img_name    = "reordered_image_w_thymus_regions_50um.ome.tif", 
                       subset      = True)

In [ ]:
img_dir = os.path.join(raw_dir, "20x_pan_DAPI_CD63_CD11c_Sirpa_A")

gating_overlay_wrapper(img_dir     = img_dir, 
                       gating_name = "combined_gated_cell_types.csv", 
                       cell_type   = "aDC2", 
                       mask_name   = "cp2_mask.tif", 
                       img_name    = "reordered_image_w_thymus_regions_50um.ome.tif",
                       subset      = True)

In [ ]:
img_dir = os.path.join(raw_dir, "20x_pan_DAPI_CD63_CD11c_Sirpa_B")

gating_overlay_wrapper(img_dir     = img_dir, 
                       gating_name = "combined_gated_cell_types.csv", 
                       cell_type   = "aDC2", 
                       mask_name   = "cp2_mask.tif", 
                       img_name    = "reordered_image_w_thymus_regions_50um.ome.tif",
                       subset      = True)

In [ ]:
img_dir = os.path.join(raw_dir, "20x_pan_DAPI_CD63_CD11c_Sirpa_D")

gating_overlay_wrapper(img_dir     = img_dir, 
                       gating_name = "combined_gated_cell_types.csv", 
                       cell_type   = "aDC2", 
                       mask_name   = "cp2_mask.tif", 
                       img_name    = "reordered_image_w_thymus_regions_50um.ome.tif",
                       subset      = True)
    ## This one has a really bad Sirpa stain.

In [ ]:
img_dir = os.path.join(raw_dir, "20x_pan_DAPI_CD207_CD11c_XCR1_A")

gating_overlay_wrapper(img_dir     = img_dir, 
                       gating_name = "combined_gated_cell_types.csv", 
                       cell_type   = "CD207+cDC1", 
                       mask_name   = "cp2_mask.tif", 
                       img_name    = "reordered_image_w_thymus_regions_50um.ome.tif",
                       subset      = True)
    ## This one is oversegmenting quite badly.

In [ ]:
img_dir = os.path.join(raw_dir, "20x_pan_DAPI_CD207_CD11c_XCR1_B")

gating_overlay_wrapper(img_dir     = img_dir, 
                       gating_name = "combined_gated_cell_types.csv", 
                       cell_type   = "CD207+cDC1", 
                       mask_name   = "cp2_mask.tif", 
                       img_name    = "reordered_image_w_thymus_regions_50um.ome.tif",
                       subset      = True)

In [ ]:
img_dir = os.path.join(raw_dir, "20x_pan_DAPI_CD207_CD11c_XCR1_D")

gating_overlay_wrapper(img_dir     = img_dir, 
                       gating_name = "combined_gated_cell_types.csv", 
                       cell_type   = "CD207+cDC1", 
                       mask_name   = "cp2_mask.tif", 
                       img_name    = "reordered_image_w_thymus_regions_50um.ome.tif",
                       subset      = True)

In [ ]:
img_dir = os.path.join(raw_dir, "20x_pan_DAPI_CD63_CD11c_XCR1_A")

gating_overlay_wrapper(img_dir     = img_dir, 
                       gating_name = "combined_gated_cell_types.csv", 
                       cell_type   = "aDC1",  
                       mask_name   = "cp2_mask.tif", 
                       img_name    = "reordered_image_w_thymus_regions_50um.ome.tif",
                       subset      = True)

In [ ]:
img_dir = os.path.join(raw_dir, "20x_pan_DAPI_CD63_CD11c_XCR1_B")

gating_overlay_wrapper(img_dir     = img_dir, 
                       gating_name = "combined_gated_cell_types.csv", 
                       cell_type   = "aDC1",  
                       mask_name   = "cp2_mask.tif", 
                       img_name    = "reordered_image_w_thymus_regions_50um.ome.tif",
                       subset      = True)

In [ ]:
img_dir = os.path.join(raw_dir, "20x_pan_DAPI_CD63_CD11c_XCR1_C")

gating_overlay_wrapper(img_dir     = img_dir, 
                       gating_name = "combined_gated_cell_types.csv", 
                       cell_type   = "aDC1",  
                       mask_name   = "cp2_mask.tif", 
                       img_name    = "reordered_image_w_thymus_regions_50um.ome.tif",
                       subset      = True)

In [ ]:
img_dir = os.path.join(raw_dir, "20x_pan_DAPI_CD63_CD11c_XCR1_D")

gating_overlay_wrapper(img_dir     = img_dir, 
                       gating_name = "combined_gated_cell_types.csv", 
                       cell_type   = "aDC1",  
                       mask_name   = "cp2_mask.tif", 
                       img_name    = "reordered_image_w_thymus_regions_50um.ome.tif",
                       subset      = True)
## This one has some capsule issues.

In [ ]:
img_dir = os.path.join(raw_dir, "20x_pan_DAPI_Sirpa_CD11c_CD14_A")

gating_overlay_wrapper(img_dir     = img_dir, 
                       gating_name = "combined_gated_cell_types.csv", 
                       cell_type   = "CD14+cDC2",
                       mask_name   = "cp2_mask.tif", 
                       img_name    = "reordered_image_w_thymus_regions_50um.ome.tif",
                       subset      = True)

In [ ]:
img_dir = os.path.join(raw_dir, "20x_pan_DAPI_Sirpa_CD11c_CD14_B")

gating_overlay_wrapper(img_dir     = img_dir, 
                       gating_name = "combined_gated_cell_types.csv", 
                       cell_type   = "CD14+cDC2",
                       mask_name   = "cp2_mask.tif", 
                       img_name    = "reordered_image_w_thymus_regions_50um.ome.tif",
                       subset      = True)

In [ ]:
img_dir = os.path.join(raw_dir, "20x_pan_DAPI_Sirpa_CD11c_CD14_C")

gating_overlay_wrapper(img_dir     = img_dir, 
                       gating_name = "combined_gated_cell_types.csv", 
                       cell_type   = "CD14+cDC2",
                       mask_name   = "cp2_mask.tif", 
                       img_name    = "reordered_image_w_thymus_regions_50um.ome.tif",
                       subset      = True)

In [ ]:
img_dir = os.path.join(raw_dir, "20x_pan_DAPI_Sirpa_CD63_MerTK_A")

gating_overlay_wrapper(img_dir     = img_dir, 
                       gating_name = "combined_gated_cell_types.csv", 
                       cell_type   = "aDC2",
                       mask_name   = "cp2_mask.tif", 
                       img_name    = "reordered_image_w_thymus_regions_50um.ome.tif",
                       subset      = True)

In [ ]:
img_dir = os.path.join(raw_dir, "20x_pan_DAPI_Sirpa_CD63_MerTK_A")

gating_overlay_wrapper(img_dir     = img_dir, 
                       gating_name = "combined_gated_cell_types.csv", 
                       cell_type   = "Macrophage",
                       mask_name   = "cp2_mask.tif", 
                       img_name    = "reordered_image_w_thymus_regions_50um.ome.tif",
                       subset      = True)

In [ ]:
img_dir = os.path.join(raw_dir, "20x_pan_DAPI_Sirpa_CD63_MerTK_B")

gating_overlay_wrapper(img_dir     = img_dir, 
                       gating_name = "combined_gated_cell_types.csv", 
                       cell_type   = "aDC2",
                       mask_name   = "cp2_mask.tif", 
                       img_name    = "reordered_image_w_thymus_regions_50um.ome.tif",
                       subset      = True)

In [ ]:
img_dir = os.path.join(raw_dir, "20x_pan_DAPI_Sirpa_CD63_MerTK_B")

gating_overlay_wrapper(img_dir     = img_dir, 
                       gating_name = "combined_gated_cell_types.csv", 
                       cell_type   = "Macrophage",
                       mask_name   = "cp2_mask.tif", 
                       img_name    = "reordered_image_w_thymus_regions_50um.ome.tif",
                       subset      = True)

In [ ]:
img_dir = os.path.join(raw_dir, "20x_pan_DAPI_Sirpa_CD63_MerTK_D")

gating_overlay_wrapper(img_dir     = img_dir, 
                       gating_name = "combined_gated_cell_types.csv", 
                       cell_type   = "aDC2",
                       mask_name   = "cp2_mask.tif", 
                       img_name    = "reordered_image_w_thymus_regions_50um.ome.tif",
                       subset      = True)